# Part 5: Coordinate Transformations & Creating Basic Maps



## Coordinate Transformations using Pyproj


<img src="https://pyproj4.github.io/pyproj/stable/_static/logo.png" style="width: 200px;"></img>

Pyproj is a Python library that provides a set of tools and utilities for working with geospatial coordinate reference systems (CRS) and coordinate transformations. It is built on top of the PROJ library, which is a widely used open-source library for geospatial transformations.

With Pyproj, you can perform various geospatial operations such as coordinate transformations: Pyproj can be used to transform coordinates between different CRS, including converting between geographic and projected CRS. This is useful for converting data between different coordinate systems, such as when working with data from different sources or when plotting data on a map.
Pyproj can also be used to perform Geodetic calculations: Pyproj provides functions for performing geodetic calculations, such as calculating distances between points on the earth's surface, calculating azimuths and bearings, and performing geodetic intersections.


https://pyproj4.github.io/pyproj/stable/api/proj.html


### EPSG

EPSG (European Petroleum Survey Group) codes are a standardized system for identifying coordinate reference systems (CRS) used in geospatial data. Each EPSG code represents a specific CRS, which defines how geographic coordinates (latitude and longitude) are projected onto a two-dimensional map or image.

The EPSG system was developed by the European Petroleum Survey Group in the 1980s, but it is now maintained by the International Association of Oil & Gas Producers (IOGP). The EPSG registry contains thousands of codes for different CRSs, covering both commonly used and obscure projections.

EPSG codes are widely used in many geospatial software applications, including GIS (Geographic Information Systems), remote sensing software, and web mapping services. They are useful for ensuring that data from different sources are correctly aligned, by specifying the correct CRS for each dataset.

For example, the EPSG code **4326** represents the WGS 84 CRS, which is a common CRS used for GPS coordinates. The EPSG code **3857** represents the Web Mercator CRS, which is widely used in web mapping applications.


Some common EPSG codes are:

* EPSG:4326 - WGS 84 - This is a geographic coordinate system commonly used for GPS data and represents the world in latitude and longitude.
* EPSG:3857 - Web Mercator - This is a projected coordinate system used by many web mapping applications, including Google Maps and OpenStreetMap.
* EPSG:26918 - UTM Zone 18N - This is a projected coordinate system commonly used in North America for mapping at a regional scale.
* EPSG:32618 - WGS 84 / UTM Zone 18N - This is a hybrid coordinate system that combines the geographic coordinate system of WGS 84 with the projected coordinate system of UTM Zone 18N.
* EPSG:3035 - ETRS89 / LAEA Europe - This is a projected coordinate system used for mapping in Europe.
* EPSG:3857 - Pseudo-Mercator - This is a variant of the Web Mercator coordinate system used by many mapping applications.
* EPSG:27700 - OSGB 1936 / British National Grid - This is a projected coordinate system used for mapping in the United Kingdom.
* EPSG:2154 - RGF93 / Lambert-93 - This is a projected coordinate system used for mapping in France.
* EPSG:4269 - NAD83 - This is a geographic coordinate system commonly used in North America for mapping at a national scale.

Also check out https://epsg.io


In [ ]:
import pyproj
from pyproj import Transformer

wgs84= "epsg:4326"
lv95= "epsg:2056"

In [ ]:
transformer1 = Transformer.from_crs("epsg:4326", "epsg:2056")

r0 = transformer1.transform(47.3771216, 8.5391632)
r0

In [ ]:
transformer2 = Transformer.from_crs("epsg:2056", "epsg:4326")

r1 = transformer2.transform(2683111.9823819078, 1247947.5735251226)
r1

## Geodesic Line

A geodesic line is the shortest path between two points on a curved surface, such as the surface of the earth. It is also known as a "great circle" or "geodesic arc."

On a flat surface, such as a piece of paper, the shortest path between two points is a straight line. However, on a curved surface like the earth, a straight line between two points is not the shortest distance. Instead, the shortest distance is along a curved line that follows the surface of the earth. This is known as a geodesic line.

For example, if you wanted to travel from New York City to London, a straight line on a map would not be the shortest path, as the earth's surface is curved. Instead, the shortest path would be along a geodesic line that follows the curvature of the earth's surface. This is why airplanes flying between New York City and London typically follow a curved path.

Geodesic lines can be calculated using geodetic calculations, which take into account the shape of the earth and the curvature of its surface. They are important in many applications, including navigation, surveying, and geodesy.

https://en.wikipedia.org/wiki/Geodesic


Example: Berlin to New York

In [ ]:
g = pyproj.Geod(ellps='WGS84')

# BCC
startlong = 13.4164159
startlat =  52.5207433

# New York
endlong = -74.001457
endlat = 40.7094328

lonlats = g.npts(startlong, startlat, endlong, endlat, 30)

lonlats = [(startlong, startlat)] + lonlats +  [(endlong, endlat)]  ## add start and end point

print(len(lonlats))
print(lonlats)



In [ ]:
import geopandas as gpd
import geoplot
from shapely.geometry import Point

geodesic = [Point(xy) for xy in zip(lonlats)]

In [ ]:
gdfGeodesic = gpd.GeoDataFrame(geometry=geodesic, crs="EPSG:4326")
gdfGeodesic.plot();

Combine it with a vector dataset of the planet

In [ ]:
gdfCountries = gpd.read_file("geodata/packages/natural_earth_vector.gpkg", 
                              layer="ne_110m_admin_0_countries", 
                              encoding="utf-8")

In [ ]:
ax = gdfCountries.plot(figsize=(20,10), facecolor="#EEEEEE", edgecolor="#000000")
gdfGeodesic.plot(ax=ax, color="#0000FF", markersize=40);

In [ ]:
# Delete Antarctica from the dataframe
world = gdfCountries.drop(gdfCountries.index[gdfCountries['NAME'] == 'Antarctica'])

In [ ]:
# Convert to Web-Mercator:
world = world.to_crs("EPSG:3857")
world.plot()

In [ ]:
ax = world.plot(figsize=(20,10), facecolor="#EEEEEE", edgecolor="#000000")
gdfGeodesic.to_crs("EPSG:3857").plot(ax=ax, color="#0000FF", markersize=40);

**Missing Antarctica ?**

Well the problem is that the Mercator / WebMercator will go to infinity near the poles. 

In [ ]:
gdfCountries.to_crs("EPSG:3857").plot()

### Solution

We can solve the problem in the same way Google Maps did it: clip the world!

In Google maps the world is a perfect sphere. Lets figure out what the max/min latitude is when you want to have a square:


Transform WGS84 to WebMercator:

In [ ]:
transformer = Transformer.from_crs("epsg:4326", "epsg:3857")

r0 = transformer.transform(0,-180)
r1 = transformer.transform(0,180)

print(r0,r1)

To make it a perfect square we take the same values for the y-axis and transform it back to geographic coordinates (WGS84)

In [ ]:
transformer = Transformer.from_crs("epsg:3857", "epsg:4326")

r0 = transformer.transform(-20037508.342789244,-20037508.342789244)
r1 = transformer.transform(20037508.342789244,20037508.342789244)

print(r0,r1)

Now clip it by creating a box in shapely

In [ ]:
from shapely.geometry import box

bbox = box(-180, r0[0], 180, r1[0])
bbox

In [ ]:
world2 = gpd.clip(gdfCountries, bbox)

In [ ]:
world2.plot();

In [ ]:
world2.to_crs("EPSG:3857").plot();

Recognize Google Maps (or OpenStreetMap, or ...) ?